# Bibliotecas

In [1]:
import numpy  as np
import pandas as pd
import pickle
import warnings

from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

warnings.filterwarnings( 'ignore' )

# Importação de dados

In [2]:
df_03 = pd.read_pickle('../exports/cicle_exports/03_filtering_to_business/df_03.pkl', compression="gzip")

In [3]:
df_05 = df_03.copy()

# Funções

In [4]:
def apply_ohe(encoder ,encoded_array, variable_name, df):
    '''
    Aplica o One Hot Encoder no dataframe, criando as colunas respectivas às categorias no encoder e elimininado a coluna original.

    Parâmetros:
        encoder (OneHotEncoder): Encoder OneHotEncoder do Scikit-learn pré-configurado.
        encoded_array (ndarray): Array gerado pelo OneHotEncoder.
        variable_name (str): Nome da variável original.
        df (DataFrame): Dataframe a ser alterado.

    Retorna:
        Um novo dataframe com as novas colunas respectivas às categorias no encoder e sem a coluna original.
    '''

    # Criar DataFrame com nomes corretos
    encoded_df = pd.DataFrame(
        encoded_array,
        columns=encoder.get_feature_names_out([variable_name]),
        index=df.index
    )

    # Concatenar com o DataFrame original, removendo a coluna original
    df_f = pd.concat([df.drop(columns=[variable_name]), encoded_df], axis=1)

    return df_f

# TVT - Separação entre treino, validação e teste (70% - 15% - 15%)

A separação será feita respeitando o ordenamento temporal, pois trata-se de um caso de **Série Temporal**.

In [5]:
# Criando dataframe com datas únicas
unique_days = df_05[['year', 'month', 'day']].drop_duplicates()
unique_days = unique_days.sort_values(['year', 'month', 'day']).reset_index(drop=True)

# Calculando o tamanho de cada divisão
n = len(unique_days)
train_end = int(n * 0.7)
val_end = train_end + int(n * 0.15)

# Obtendo as semanas para cada divisão
train_days = unique_days.iloc[:train_end]
val_days = unique_days.iloc[train_end:val_end]
test_days = unique_days.iloc[val_end:]

# Criando os dataframes com base nas semanas
def filter_days(df_05, days):
    return df_05.merge(days, on=['year', 'month', 'day'], how='inner')

df_05_train = filter_days(df_05, train_days)
df_05_val = filter_days(df_05, val_days)
df_05_test = filter_days(df_05, test_days)

# 5.0. Prepararação dos dados

## 5.1. Normalizacao

Não há dados a serem normalizados, pois não há variáveis numéricas que possuem um comportamento de distribuição normal (gausiano). Vide passo 4 - análise exploratória.

## 5.2. Rescaling

In [6]:
# competition_distance
rs_competition_distance = RobustScaler() # Reescalador tratador de outliers
df_05_train['competition_distance'] = rs_competition_distance.fit_transform( df_05_train[['competition_distance']].values )

# competition_time_month
rs_competition_time_month = RobustScaler() # Reescalador tratador de outliers
df_05_train['competition_time_month'] = rs_competition_time_month.fit_transform( df_05_train[['competition_time_month']].values )

# promo_time_week
mms_promo_time_week = MinMaxScaler() # Reescalador normal
df_05_train['promo_time_week'] = mms_promo_time_week.fit_transform( df_05_train[['promo_time_week']].values )

# year
mms_year = MinMaxScaler() # Reescalador normal
df_05_train['year'] = mms_year.fit_transform( df_05_train[['year']].values )


## 5.3. Transformação

### 5.3.1. Encoding

In [7]:
# state_holiday - One Hot Encoding
ohe_state_holiday = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_array = ohe_state_holiday.fit_transform(df_05_train[['state_holiday']])
df_05_train = apply_ohe(ohe_state_holiday, encoded_array, 'state_holiday', df_05_train)

# store_type - Label Encoding
le_store_type = LabelEncoder()
df_05_train['store_type'] = le_store_type.fit_transform( df_05_train[['store_type']].values )

# assortment - Ordinal Encoding
assortment_dict = {'basic': 1,  'extra': 2, 'extended': 3}
df_05_train['assortment'] = df_05_train['assortment'].map( assortment_dict )

### 5.3.2. Transformação da variável resposta

In [8]:
# Aplicando escala logarítimica:
df_05_train['sales'] = np.log1p( df_05_train['sales'] )

### 5.3.3. Transformação de natureza (encoder cíclico)

In [9]:
# day of week
df_05_train['day_of_week_sin'] = df_05_train['day_of_week'].apply( lambda x: np.sin( x * ( 2. * np.pi/7 ) ) )
df_05_train['day_of_week_cos'] = df_05_train['day_of_week'].apply( lambda x: np.cos( x * ( 2. * np.pi/7 ) ) )

# month
df_05_train['month_sin'] = df_05_train['month'].apply( lambda x: np.sin( x * ( 2. * np.pi/12 ) ) )
df_05_train['month_cos'] = df_05_train['month'].apply( lambda x: np.cos( x * ( 2. * np.pi/12 ) ) )

# day 
df_05_train['day_sin'] = df_05_train['day'].apply( lambda x: np.sin( x * ( 2. * np.pi/30 ) ) )
df_05_train['day_cos'] = df_05_train['day'].apply( lambda x: np.cos( x * ( 2. * np.pi/30 ) ) )

# week of year
df_05_train['week_of_year_sin'] = df_05_train['week_of_year'].apply( lambda x: np.sin( x * ( 2. * np.pi/52 ) ) )
df_05_train['week_of_year_cos'] = df_05_train['week_of_year'].apply( lambda x: np.cos( x * ( 2. * np.pi/52 ) ) )

## 5.4. Descartando colunas antigas

In [10]:
cols_drop = ['week_of_year', 'day', 'month', 'day_of_week', 'promo_since', 'competition_since', 'year_week' ]
df_05_train = df_05_train.drop( cols_drop, axis=1 )

## 5.5. Aplicando data preparation nos dados de validação e teste

In [11]:
def apply_data_preparation(df: pd.DataFrame):
    """
    Aplica todas as transformações da etapa de data preparation no dataframe inserido.

    Args:
        df (pd.DataFrame): DataFrame para aplicar transformações.

    """
    # Normalização - Não há dados

    df_f = df.copy()

    # Rescaling

    # competition_distance
    df_f['competition_distance'] = rs_competition_distance.transform( df_f[['competition_distance']].values )

    # competition_time_month
    df_f['competition_time_month'] = rs_competition_time_month.transform( df_f[['competition_time_month']].values )

    # promo_time_week
    df_f['promo_time_week'] = mms_promo_time_week.transform( df_f[['promo_time_week']].values )

    # year
    df_f['year'] = mms_year.transform( df_f[['year']].values )


    # Encoding

    # state_holiday - One Hot Encoding
    encoded_array = ohe_state_holiday.transform(df_f[['state_holiday']])
    df_f = apply_ohe(ohe_state_holiday, encoded_array, 'state_holiday', df_f)

    # store_type - Label Encoding
    df_f['store_type'] = le_store_type.transform( df_f[['store_type']].values )

    # assortment - Ordinal Encoding
    assortment_dict = {'basic': 1,  'extra': 2, 'extended': 3}
    df_f['assortment'] = df_f['assortment'].map( assortment_dict )


    # Transformação da variável resposta

    # Aplicando escala logarítimica:
    df_f['sales'] = np.log1p( df_f['sales'] )


    # Transformação de natureza (encoder cíclico)

    # day of week
    df_f['day_of_week_sin'] = df_f['day_of_week'].apply( lambda x: np.sin( x * ( 2. * np.pi/7 ) ) )
    df_f['day_of_week_cos'] = df_f['day_of_week'].apply( lambda x: np.cos( x * ( 2. * np.pi/7 ) ) )

    # month
    df_f['month_sin'] = df_f['month'].apply( lambda x: np.sin( x * ( 2. * np.pi/12 ) ) )
    df_f['month_cos'] = df_f['month'].apply( lambda x: np.cos( x * ( 2. * np.pi/12 ) ) )

    # day 
    df_f['day_sin'] = df_f['day'].apply( lambda x: np.sin( x * ( 2. * np.pi/30 ) ) )
    df_f['day_cos'] = df_f['day'].apply( lambda x: np.cos( x * ( 2. * np.pi/30 ) ) )

    # week of year
    df_f['week_of_year_sin'] = df_f['week_of_year'].apply( lambda x: np.sin( x * ( 2. * np.pi/52 ) ) )
    df_f['week_of_year_cos'] = df_f['week_of_year'].apply( lambda x: np.cos( x * ( 2. * np.pi/52 ) ) )


    # Descartando colunas antigas

    cols_drop = ['week_of_year', 'day', 'month', 'day_of_week', 'promo_since', 'competition_since', 'year_week' ]
    df_f = df_f.drop( cols_drop, axis=1 )

    return df_f

In [12]:
df_05_val = apply_data_preparation(df_05_val)
df_05_test = apply_data_preparation(df_05_test)

# Exportação de resultados

In [13]:
df_05_train.to_pickle   ('../exports/cicle_exports/05_data_preparation/df_05_train.pkl', compression='gzip')
df_05_val.to_pickle     ('../exports/cicle_exports/05_data_preparation/df_05_val.pkl', compression='gzip')
df_05_test.to_pickle    ('../exports/cicle_exports/05_data_preparation/df_05_test.pkl', compression='gzip')

**Exportação para produto final:**

Todos os encoders e scalers que necessitam de um processo de fit serão exportados para não haver necessidade de repetir o processo de fit durante a produção. Portanto esses arquivos farão parte do produto final.

In [14]:
pickle.dump(rs_competition_distance,    open( '../exports/cicle_products/rs_competition_distance.pkl', 'wb'))
pickle.dump(rs_competition_time_month,  open( '../exports/cicle_products/rs_competition_time_month.pkl', 'wb'))

pickle.dump(mms_promo_time_week,        open( '../exports/cicle_products/mms_promo_time_week.pkl', 'wb'))
pickle.dump(mms_year,                   open( '../exports/cicle_products/mms_year.pkl', 'wb'))

pickle.dump(ohe_state_holiday,          open( '../exports/cicle_products/ohe_state_holiday.pkl', 'wb'))

pickle.dump(le_store_type,              open( '../exports/cicle_products/le_store_type.pkl', 'wb'))